In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
#print(np.random.seed(123))

#The numpy.random.seed() makes the
#random numbers predictable and is used for reproducibility

import os
import glob

"""crea entorno grÃ¡fico"""

plt.rcParams['figure.figsize'] = (40, 50)
plt.style.use('ggplot')

# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
os.chdir("..\\Datasets")
os.getcwd()
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
#print(csv_files)
list_data = []
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

df = pd.concat(list_data,ignore_index=True)
#tipos=df.dtypes  #tipos de datos de campos

"""
""" 

"""información datos"""
#print(df.head(5))
#print('Cantidad de Filas y columnas:',df.shape)
#print('Nombre columnas:',df.columns)
"""Columnas, nulos y tipo de datos"""
#print(df.info())
"""descripción estadística de los datos numéricos"""
#print(df.describe())

"""PREPARO DATASET DF2"""

df2=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3=df[['Label']]
df2['Label']=df3[['Label']]

"""PREPARO DATASET DF2_LR"""

df2_LR=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3_LR=df[['Label']]
df2_LR['Label']=df3_LR[['Label']]


"""desordena el dataset"""

df2=df2.sort_values('Fwd_Pkt_Len_Min')
df2 = df2[df2['Label'].isin([1,2,3,4])]

df2_LR=df2_LR.sort_values('Fwd_Pkt_Len_Min')
df2_LR = df2_LR[df2_LR['Label'].isin([1,2,3,4])]


""" convertir valores número la clase"""
df2['Label'] = df2['Label'].replace(1,"Benigno")
df2['Label'] = df2['Label'].replace(2,"Malware")
df2['Label'] = df2['Label'].replace(3,"Malware")
df2['Label'] = df2['Label'].replace(4,"Malware")

df2_LR['Label'] = df2_LR['Label'].replace(1,"Benigno")
df2_LR['Label'] = df2_LR['Label'].replace(2,"Malware")
df2_LR['Label'] = df2_LR['Label'].replace(3,"Malware")
df2_LR['Label'] = df2_LR['Label'].replace(4,"Malware")
#print(df2.head(10))
#print(df2.groupby('Label').size())

""" balanceo el dataset entre benigno y malware"""
dff=df2[:]
dff1 = dff[dff.Label == 'Malware']
dff = dff.drop(dff[dff['Label'] == 'Malware'].index)
a = dff1.iloc[0:134800,]
df2 = pd.concat([dff,a])

dff_LR=df2_LR[:]
dff1_LR = dff_LR[dff_LR.Label == 'Malware']
dff_LR = dff_LR.drop(dff_LR[dff_LR['Label'] == 'Malware'].index)
a_LR = dff1_LR.iloc[0:134800,]
df2_LR = pd.concat([dff_LR,a_LR])

"""agrupa por nombres"""
#print(df2.groupby('Label').size())
#print(df2_LR.groupby('Label').size())

""" convierto valor nombre a 0=benigno 1=malware"""

label_encoder = LabelEncoder()
df2.iloc[:,15] = label_encoder.fit_transform(df2.iloc[:,15]).astype('int64')


label_encoder = LabelEncoder()
df2_LR.iloc[:,15] = label_encoder.fit_transform(df2_LR.iloc[:,15]).astype('int64')

"""agrupa por 0 y 1 """

#print(df2.groupby('Label').size())
#print(df2_LR.groupby('Label').size())


"""obtengo variables con correlación entre los datos"""
#import seaborn as sns
#f_LR, ax_LR = plt.subplots(figsize=(10, 8))
corr_LR = df2_LR.corr()
#sns.heatmap(corr_LR, mask=np.zeros_like(corr_LR, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#            square=True, ax=ax_LR)

"""elimino variables correlacionadas"""
			
columns_LR = np.full((corr_LR.shape[0],), True, dtype=bool)
for i in range(corr_LR.shape[0]):
    for j in range(i+1, corr_LR.shape[0]):
        if corr_LR.iloc[i,j] >= 0.9:
            if columns_LR[j]:
                columns_LR[j] = False

selected_columns_LR = df2_LR.columns[columns_LR]
df2_LR = df2_LR[selected_columns_LR]

columns = [True, True, True, True, True,
		   True, True, True, True, True,
		   True, True, True, True, True,True]

selected_columns = df2.columns[columns]



""" agupar despues de correlacionar y eliminar variables"""

#print(df2.groupby('Label').size())
#print(df2_LR.groupby('Label').size())

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 100)
pd.set_option('display.float_format', '{:.5f}'.format)

X_LR = np.array(df2_LR.drop(['Label'],1))
y_LR = np.array(df2_LR['Label'])

#X = np.array(df2.drop(['Label'],1))
#y = np.array(df2['Label'])

df2.rename(columns={'Flow_Byts/s':'Flow_Byts'}, inplace=True)

import statsmodels.api as sm
import statsmodels.formula.api as smf
model = smf.logit("""Label ~ Flow_Duration + Tot_Fwd_Pkts
				  + Tot_Bwd_Pkts + TotLen_Fwd_Pkts
				  + TotLen_Bwd_Pkts + Fwd_Pkt_Len_Min
				  + Bwd_Pkt_Len_Min + Fwd_Pkt_Len_Max
				  + Bwd_Pkt_Len_Max + Fwd_Pkt_Len_Mean
				  + Bwd_Pkt_Len_Mean + Fwd_Pkt_Len_Std
                  + Bwd_Pkt_Len_Std
				  + Flow_Byts""", data=df2)
results = model.fit()
#print(results.summary())


#ahora con dataframe completo
feature_cols = ['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts']

X = df2[feature_cols]
y = df2.Label

model = sm.Logit(y, sm.add_constant(X))
results = model.fit()
print(results.summary())

coefs = pd.DataFrame({
    'coef': results.params.values,
    'odds ratio': np.exp(results.params.values),
    'pvalue': results.pvalues,
    'name': results.params.index
})#.sort_values(by='pvalue', ascending=False)
	
print(coefs)

#============================================
#============================================
#===========================================

print(coefs[coefs.pvalue < 0.05])


#============================================
#============================================
#===========================================

feature_cols = coefs[coefs.pvalue < 0.05].drop('const').index
print(feature_cols)

#============================================
#============================================
#===========================================

print(results.pvalues.drop('const'))
print(results.pvalues.drop('const') < 0.05)


#============================================
#============================================
#===========================================


print(X.columns)
print(X.columns[results.pvalues.drop('const') < 0.05])

#============================================
#============================================
#===========================================



# Filter our original features to only keep ones with a p-value less than 0.1
feature_cols = X.columns[results.pvalues.drop('const') < 0.05]

# Run our regression just like we normally do
X = df2[feature_cols]
y = df2.Label

model = sm.Logit(y, sm.add_constant(X))
results = model.fit()
print(results.summary())

# Filter our original features to only keep ones with a p-value less than 0.05
feature_cols = X.columns[results.pvalues.drop('const') < 0.05]

# Run our regression just like we normally do
X = df2[feature_cols]
y = df2.Label

model = sm.Logit(y, sm.add_constant(X))
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: inf
         Iterations 9
Optimization terminated successfully.
         Current function value: inf
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  Label   No. Observations:               272707
Model:                          Logit   Df Residuals:                   272691
Method:                           MLE   Df Model:                           15
Date:                Fri, 19 Nov 2021   Pseudo R-squ.:                    -inf
Time:                        11:20:23   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                   -1.8901e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
cons

Optimization terminated successfully.
         Current function value: inf
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  Label   No. Observations:               272707
Model:                          Logit   Df Residuals:                   272693
Method:                           MLE   Df Model:                           13
Date:                Fri, 19 Nov 2021   Pseudo R-squ.:                    -inf
Time:                        11:20:30   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                   -1.8901e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.4324      0.008    -52.511      0.000      -0.449      -0.416
Flow_Duration  